# Explanatio of Good results by Ichiki

## Import

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
import xgboost as xgb
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression


/home/kazuki/libraries/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Data preprocessing

In [3]:
#Load data
##train data
print('Load data...')
train = pd.read_csv("../../data/bank_satis/train/train.csv")
##Get trainID
train_id = train['ID'].values
##Get Answer from TARGET column
target = train['TARGET'].values
##drop needles values(ID, TARGET) ----> only features remain
train = train.drop(['ID','TARGET'],axis=1)

##test data
test = pd.read_csv("../../data/bank_satis/test/test.csv")
test_row = pd.read_csv("../../data/bank_satis/test/test.csv")
##Get testID
test_id = test['ID'].values
##Drop values(ID) --->only features remain
test = test.drop(['ID'],axis=1)

train.head()

Load data...


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40532.10
1,5,2,35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45486.72
2,6,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46993.95
3,7,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187898.61
4,9,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73649.73


In [4]:
#train features(=train columns' length)=test features(369=369)
test.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40532.10
1,2,35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45486.72
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46993.95
3,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187898.61
4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73649.73


### Var3

In [5]:
#Consider var3(Nationality)
#-999999 may be no answer to the question
#-999999 should be median value(=2)
train.var3.value_counts()

 2         74165
 8           138
-999999      116
 9           110
 3           108
 1           105
 13           98
 7            97
 4            86
 12           85
 6            82
 0            75
 10           72
 11           66
 5            63
 14           61
 15           34
 18           10
 16            9
 17            7
 23            7
 25            6
 142           6
 154           6
 20            6
 31            6
 38            6
 153           5
 24            5
 91            5
           ...  
 225           1
 169           1
 41            1
 228           1
 101           1
 165           1
 229           1
 168           1
 108           1
 238           1
 184           1
 112           1
 76            1
 77            1
 205           1
 119           1
 79            1
 182           1
 181           1
 210           1
 40            1
 211           1
 84            1
 213           1
 178           1
 177           1
 87            1
 151          

In [6]:
#removing outliers
train = train.replace(-999999,2)
test = test.replace(-999999,2)

#check values
train.var3.value_counts()[:3]

2    74281
8      138
9      110
Name: var3, dtype: int64

In [7]:
test.var3.value_counts()[:3]

2    74082
8      116
9      108
Name: var3, dtype: int64

### add zero counts

In [8]:
# adding zero counts

train["zeroes"] = (train == 0).astype(int).sum(axis=1)
test["zeroes"] = (test == 0).astype(int).sum(axis=1)

In [9]:
#check values
train.zeroes.head()

0    355
1    329
2    340
3    309
4    319
Name: zeroes, dtype: int64

In [10]:
test.zeroes.head()

0    338
1    341
2    322
3    355
4    340
Name: zeroes, dtype: int64

In [11]:
#example) id0 is 355times of 0.
#this is 355 of 370values...!
train.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,zeroes
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,355
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,329
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,340
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,309
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,319


### remove constant columns

In [12]:
#present train and test shape
print(train.shape, test.shape)


((76020, 370), (75818, 370))


In [13]:
# remove constant columns (std = 0)
remove = []
for col in train.columns:
    if train[col].std() == 0:
        remove.append(col)

train.drop(remove, axis=1, inplace=True)
test.drop(remove, axis=1, inplace=True)

print(train.shape, test.shape)

((76020, 336), (75818, 336))


### remove duplicated columns

In [14]:
#present train and test shape
print(train.shape, test.shape)

((76020, 336), (75818, 336))


In [15]:
# remove duplicated columns
remove = []
cols = train.columns
for i in range(len(cols)-1):
    v = train[cols[i]].values
    for j in range(i+1,len(cols)):
        if np.array_equal(v,train[cols[j]].values):
            remove.append(cols[j])

train.drop(remove, axis=1, inplace=True)
test.drop(remove, axis=1, inplace=True)
print(train.shape, test.shape)

((76020, 307), (75818, 307))


## Feature selection 1

In [16]:
#memo:pandas DataFrame to List -->1) .values to numpy arran then 2) .tolist to List
#Cols is the List of columns names 
Cols = train.columns.values.tolist()
#memo:sklearn 
clf = GradientBoostingClassifier(random_state=1729)
#memo:The type of train and target is PD.DataFrame, can this use fit??
selector = clf.fit(train, target)
#Search important features
#importaces len is 307(sorted importance)
importances = selector.feature_importances_
#memo:SelectFromModel(estimator,threshhold, prefit)
fs = SelectFromModel(selector, prefit=True)
train = fs.transform(train)
test = fs.transform(test)
print(train.shape, test.shape)

((76020, 52), (75818, 52))


In [17]:
#selectedCols= the number of selectcols(52)
selectedCols = train.shape[1]

#take the soreted features name and values 
sortedCols = [col for importance, col  in sorted(zip(importances, Cols))]
sortedCols = sortedCols[0:selectedCols]

len(sortedCols)

52

In [18]:
#what are choosed features?
sortedCols

['delta_imp_amort_var18_1y3',
 'delta_imp_amort_var34_1y3',
 'delta_imp_aport_var13_1y3',
 'delta_imp_aport_var17_1y3',
 'delta_imp_aport_var33_1y3',
 'delta_imp_compra_var44_1y3',
 'delta_imp_reemb_var13_1y3',
 'delta_imp_reemb_var17_1y3',
 'delta_imp_reemb_var33_1y3',
 'delta_imp_trasp_var17_in_1y3',
 'delta_imp_trasp_var17_out_1y3',
 'delta_imp_trasp_var33_in_1y3',
 'delta_imp_trasp_var33_out_1y3',
 'delta_imp_venta_var44_1y3',
 'delta_num_aport_var13_1y3',
 'delta_num_aport_var17_1y3',
 'delta_num_aport_var33_1y3',
 'delta_num_compra_var44_1y3',
 'delta_num_venta_var44_1y3',
 'imp_amort_var18_ult1',
 'imp_amort_var34_ult1',
 'imp_aport_var13_hace3',
 'imp_aport_var13_ult1',
 'imp_aport_var17_hace3',
 'imp_aport_var17_ult1',
 'imp_aport_var33_hace3',
 'imp_aport_var33_ult1',
 'imp_compra_var44_hace3',
 'imp_op_var40_comer_ult1',
 'imp_op_var40_comer_ult3',
 'imp_op_var41_comer_ult3',
 'imp_reemb_var17_hace3',
 'imp_reemb_var33_ult1',
 'imp_trasp_var17_in_hace3',
 'imp_trasp_var17_in

In [19]:
train = pd.DataFrame(train)
test = pd.DataFrame(test)
train.columns = sortedCols
test.columns = sortedCols

In [20]:
train.head()

,delta_imp_amort_var18_1y3,delta_imp_amort_var34_1y3,delta_imp_aport_var13_1y3,delta_imp_aport_var17_1y3,delta_imp_aport_var33_1y3,delta_imp_compra_var44_1y3,delta_imp_reemb_var13_1y3,delta_imp_reemb_var17_1y3,delta_imp_reemb_var33_1y3,delta_imp_trasp_var17_in_1y3,...,ind_var10_ult1,ind_var10cte_ult1,ind_var12,ind_var12_0,ind_var13,ind_var13_0,ind_var13_corto,ind_var13_corto_0,ind_var13_largo,ind_var13_largo_0
0,2.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,39205.170000,355.0
1,2.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,88.89,0.00,0.00,0.0,0.0,0.0,49278.030000,329.0
2,2.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.00,0.18,3.00,2.07,0.0,0.0,0.0,67333.770000,340.0
3,2.0,37.0,0.0,0.0,0.0,0.0,195.0,0.0,0.0,195.0,...,18.0,186.09,0.00,91.56,138.84,0.0,0.0,0.0,64007.970000,309.0
4,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.00,0.30,40501.08,13501.47,0.0,0.0,0.0,117310.979016,319.0


In [21]:
test.head()

,delta_imp_amort_var18_1y3,delta_imp_amort_var34_1y3,delta_imp_aport_var13_1y3,delta_imp_aport_var17_1y3,delta_imp_aport_var33_1y3,delta_imp_compra_var44_1y3,delta_imp_reemb_var13_1y3,delta_imp_reemb_var17_1y3,delta_imp_reemb_var33_1y3,delta_imp_trasp_var17_in_1y3,...,ind_var10_ult1,ind_var10cte_ult1,ind_var12,ind_var12_0,ind_var13,ind_var13_0,ind_var13_corto,ind_var13_corto_0,ind_var13_largo,ind_var13_largo_0
0,2.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,2.43,6.00,4.80,0.0,0.0,0.0,40532.10,338.0
1,2.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,2.55,3.00,2.85,0.0,0.0,0.0,45486.72,341.0
2,2.0,23.0,0.0,0.0,60.0,60.0,60.0,60.0,60.0,60.0,...,0.0,90.0,57.00,51.45,66.15,0.0,0.0,0.0,46993.95,322.0
3,2.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,187898.61,355.0
4,2.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,30.0,3.87,30.00,21.30,0.0,0.0,0.0,73649.73,340.0


## Add features  relation (Xi-Xj)/Xi

In [22]:

#add (featurei - featurej) /featurei

for i in range(len(sortedCols)):
	for j in range(len(sortedCols)):
		colName = sortedCols[i]+"_SUBTRACT_"+sortedCols[j]+"DIVIDE"+sortedCols[i]
		train[colName] = (train[sortedCols[i]]-train[sortedCols[j]])/train[sortedCols[i]]
		test[colName] = (test[sortedCols[i]]-test[sortedCols[j]])/test[sortedCols[i]]

train = train.replace(np.inf, 999999)
train = train.replace(-np.inf, -999999)
train = train.replace(np.nan, -1)
test = test.replace(np.inf, 999999)
test = test.replace(-np.inf, -999999)
test = test.replace(np.nan, -1)

In [23]:
train.head()

,delta_imp_amort_var18_1y3,delta_imp_amort_var34_1y3,delta_imp_aport_var13_1y3,delta_imp_aport_var17_1y3,delta_imp_aport_var33_1y3,delta_imp_compra_var44_1y3,delta_imp_reemb_var13_1y3,delta_imp_reemb_var17_1y3,delta_imp_reemb_var33_1y3,delta_imp_trasp_var17_in_1y3,...,ind_var13_largo_0_SUBTRACT_ind_var10_ult1DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var10cte_ult1DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var12DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var12_0DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var13DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var13_0DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var13_cortoDIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var13_corto_0DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var13_largoDIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var13_largo_0DIVIDEind_var13_largo_0
0,2.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,-109.437099,0.0
1,2.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.000000,0.729818,1.000000,1.000000,1.0,1.0,1.0,-148.781246,0.0
2,2.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,0.991176,0.999471,0.991176,0.993912,1.0,1.0,1.0,-197.040500,0.0
3,2.0,37.0,0.0,0.0,0.0,0.0,195.0,0.0,0.0,195.0,...,0.941748,0.397767,1.000000,0.703689,0.550680,1.0,1.0,1.0,-206.145534,0.0
4,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,0.990596,0.999060,-125.962633,-41.324357,1.0,1.0,1.0,-366.746016,0.0


In [24]:
test.head()

,delta_imp_amort_var18_1y3,delta_imp_amort_var34_1y3,delta_imp_aport_var13_1y3,delta_imp_aport_var17_1y3,delta_imp_aport_var33_1y3,delta_imp_compra_var44_1y3,delta_imp_reemb_var13_1y3,delta_imp_reemb_var17_1y3,delta_imp_reemb_var33_1y3,delta_imp_trasp_var17_in_1y3,...,ind_var13_largo_0_SUBTRACT_ind_var10_ult1DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var10cte_ult1DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var12DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var12_0DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var13DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var13_0DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var13_cortoDIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var13_corto_0DIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var13_largoDIVIDEind_var13_largo_0,ind_var13_largo_0_SUBTRACT_ind_var13_largo_0DIVIDEind_var13_largo_0
0,2.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.982249,0.992811,0.982249,0.985799,1.0,1.0,1.0,-118.917456,0.0
1,2.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.991202,0.992522,0.991202,0.991642,1.0,1.0,1.0,-132.392141,0.0
2,2.0,23.0,0.0,0.0,60.0,60.0,60.0,60.0,60.0,60.0,...,1.0,0.720497,0.822981,0.840217,0.794565,1.0,1.0,1.0,-144.943944,0.0
3,2.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,-528.291859,0.0
4,2.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.911765,0.988618,0.911765,0.937353,1.0,1.0,1.0,-215.616853,0.0


## Feature Selection 2

In [25]:
#ROUND TWO
#Cols is the List of columns names 
Cols = train.columns.values.tolist()
clf = GradientBoostingClassifier(random_state=1729)
selector = clf.fit(train, target)
importances = selector.feature_importances_
fs = SelectFromModel(selector, prefit=True)
train = fs.transform(train)
test = fs.transform(test)
print(train.shape, test.shape)

((76020, 313), (75818, 313))


In [26]:
#get the number of features
selectedCols = train.shape[1]
sortedCols = [col for importance, col  in sorted(zip(importances, Cols))]
sortedCols = sortedCols[0:selectedCols]
print(sortedCols)

['delta_imp_amort_var18_1y3', 'delta_imp_amort_var18_1y3_SUBTRACT_delta_imp_amort_var18_1y3DIVIDEdelta_imp_amort_var18_1y3', 'delta_imp_amort_var18_1y3_SUBTRACT_delta_imp_aport_var13_1y3DIVIDEdelta_imp_amort_var18_1y3', 'delta_imp_amort_var18_1y3_SUBTRACT_delta_imp_aport_var17_1y3DIVIDEdelta_imp_amort_var18_1y3', 'delta_imp_amort_var18_1y3_SUBTRACT_delta_imp_compra_var44_1y3DIVIDEdelta_imp_amort_var18_1y3', 'delta_imp_amort_var18_1y3_SUBTRACT_delta_imp_reemb_var17_1y3DIVIDEdelta_imp_amort_var18_1y3', 'delta_imp_amort_var18_1y3_SUBTRACT_delta_imp_reemb_var33_1y3DIVIDEdelta_imp_amort_var18_1y3', 'delta_imp_amort_var18_1y3_SUBTRACT_delta_imp_trasp_var17_out_1y3DIVIDEdelta_imp_amort_var18_1y3', 'delta_imp_amort_var18_1y3_SUBTRACT_delta_imp_trasp_var33_in_1y3DIVIDEdelta_imp_amort_var18_1y3', 'delta_imp_amort_var18_1y3_SUBTRACT_delta_imp_trasp_var33_out_1y3DIVIDEdelta_imp_amort_var18_1y3', 'delta_imp_amort_var18_1y3_SUBTRACT_delta_imp_venta_var44_1y3DIVIDEdelta_imp_amort_var18_1y3', 'delta_i

## Fit and Predict(estimator = XGB, cv)

In [27]:
predictedResult = np.zeros(train.shape[0])
kf = KFold(train.shape[0], n_folds=10)
testPred = []
for trainIndex, testIndex in kf:
    trainFold, testFold = train[trainIndex], train[testIndex]
    trainFoldTarget, testFoldTarget = target[trainIndex], target[testIndex]
    xgbc = xgb.XGBClassifier(n_estimators = 560,learning_rate = 0.0202047,max_depth = 5,subsample = 0.6815,colsample_bytree = 0.701)
    xgbc.fit(trainFold,trainFoldTarget)
    xgbpred =xgbc.predict_proba(testFold)[:,1]
    testPred.append(xgbc.predict_proba(test)[:,1])
    predictedResult[testIndex] = xgbpred
    print(roc_auc_score(testFoldTarget, xgbpred))

print(roc_auc_score(target, predictedResult))
testPred = np.average(np.array(testPred), axis =0)

0.832426981436
0.837814569536
0.826790250389
0.832775478546
0.844231100328
0.84530069178
0.845330948471
0.868062576704
0.849562688984
0.832449790117
0.840912932667


In [67]:
#Under23 is 0
#Get Under23 DataFrame
under23 = test_row[test_row['var15'] < 23]
under23.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
67,121,2,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,198237.330000
105,243,2,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79007.070000
132,286,2,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016
137,297,2,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016
301,604,2,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177721.350000


In [68]:
#Get List Under23 ID
under23 = under23.ID.values.tolist()
under23

[121,
 243,
 286,
 297,
 604,
 698,
 752,
 866,
 946,
 992,
 1413,
 1608,
 1780,
 1810,
 1839,
 1969,
 1994,
 2144,
 2177,
 2200,
 2216,
 2315,
 2494,
 2544,
 2545,
 2684,
 3005,
 3007,
 3047,
 3366,
 3410,
 3565,
 3588,
 4024,
 4116,
 4159,
 4277,
 4333,
 4355,
 4936,
 5049,
 5540,
 5863,
 5898,
 5943,
 6221,
 6779,
 6809,
 6877,
 6896,
 7132,
 7573,
 7647,
 7757,
 7811,
 8035,
 8080,
 8106,
 8212,
 8262,
 8423,
 8470,
 8624,
 8690,
 8807,
 8942,
 8948,
 9081,
 9293,
 9814,
 10134,
 10177,
 10301,
 10413,
 10534,
 10549,
 10675,
 11085,
 11132,
 11291,
 11388,
 11422,
 11622,
 11639,
 11717,
 11739,
 11882,
 11899,
 12311,
 12359,
 12530,
 12766,
 12784,
 12819,
 12838,
 12880,
 12922,
 13467,
 13559,
 13802,
 13931,
 14150,
 14151,
 14172,
 14232,
 14264,
 14351,
 14381,
 14476,
 14483,
 14607,
 14647,
 14832,
 14926,
 14987,
 15223,
 15282,
 15303,
 15400,
 15419,
 15435,
 15473,
 15498,
 15565,
 15797,
 15876,
 15909,
 16037,
 16100,
 16187,
 16189,
 16271,
 16285,
 16922,
 16947,


In [69]:
for idx in test_row.index:
    if test_row.ix[idx, 'var15'] < 23:
        testPred[idx]=0

In [70]:
sbm_df = pd.DataFrame({"ID": test_id, "TARGET": testPred})

In [71]:
sbm_df.to_csv('submission_best_23cut',index=False)